In [12]:
import requests
import re
import json
import pandas as pd
import time
from urllib.parse import urlparse
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import os

In [13]:
# 1. Obtener el HTML de la página del ranking
load_dotenv()
url = os.getenv("TARGET_URL_INVESTMENT_GROWTH")
BASE_URL = os.getenv("BASE_URL")

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36"
}

response = requests.get(url, headers=headers)
response.raise_for_status()  # Lanza error si la petición falla

In [15]:
# 2. Parsear HTML con BeautifulSoup
soup = BeautifulSoup(response.text, "html.parser")

In [39]:
# 3. Buscar la tabla principal y extraer los links
table = soup.find("table")
country_links = []


if table:
    rows = table.find_all("tr")
    for row in rows:
        cell = row.find("td")
        if cell and cell.a:
            relative_link = cell.a["href"]
            full_url = f"{BASE_URL}{relative_link}"
            country_links.append(full_url)

# Convertir la lista en DataFrame
df_links = pd.DataFrame(country_links, columns=["url"])

# Guardar como archivo Parquet
df_links.to_parquet("data/country_investment_growth_links.parquet", index=False)

In [18]:
# 4. Mostrar resultado
print("Links encontrados:", df_links.size)

Links encontrados: 100


In [19]:
# configuracion necesaria para descargar datos

master_df = pd.DataFrame()

pattern = r'arrayToDataTable\((\[\[.*?\]\])\)'

sleep_duration = 5

In [20]:
# Cargar los países válidos desde el archivo parquet
df_countries = pd.read_parquet("data/countries_with_hpi.parquet")
valid_countries = set(df_countries["country"].unique())

In [22]:
start_time = time.time()
for url in country_links:
    # Extraer país desde el URL
    country = urlparse(url).path.strip("/").split("/")[0]

    # Verificar si el país está en la lista válida
    if country not in valid_countries:
        print(f"⏭️ País {country} no está en la lista, se omite.")
        continue

    try:
        response = requests.get(url, headers=headers)
        html = response.text

        match = re.search(pattern, html, re.DOTALL)
        if match:
            data_str = match.group(1)
            data = json.loads(data_str)

            df = pd.DataFrame(data[1:], columns=data[0])
            df.columns = ["Period", "Value"]
            df["Country"] = country

            df_pivot = df.pivot(index="Period", columns="Country", values="Value")
            master_df = pd.merge(master_df, df_pivot, how="outer", left_index=True, right_index=True)

            print(f"✅ Datos extraídos correctamente para {country}")
        else:
            print(f"⚠️ No se encontraron datos en {url}")
    except Exception as e:
        print(f"❌ Error procesando {url}: {e}")
    
    time.sleep(sleep_duration)

end_time = time.time()
total_sleep_time = len(valid_countries) * sleep_duration
effective_time = end_time - start_time - total_sleep_time

print(f"⏳ Tiempo total (incluyendo esperas): {end_time - start_time:.2f} s")
print(f"😴 Tiempo en espera: {total_sleep_time:.2f} s")
print(f"⚡ Tiempo efectivo de ejecución: {effective_time:.2f} s")

⏭️ País Albania no está en la lista, se omite.
⏭️ País Algeria no está en la lista, se omite.
⏭️ País Argentina no está en la lista, se omite.
⏭️ País Armenia no está en la lista, se omite.
✅ Datos extraídos correctamente para Australia
✅ Datos extraídos correctamente para Austria
⏭️ País Azerbaijan no está en la lista, se omite.
⏭️ País Bahamas no está en la lista, se omite.
⏭️ País Belarus no está en la lista, se omite.
✅ Datos extraídos correctamente para Belgium
⏭️ País Bermuda no está en la lista, se omite.
⏭️ País Bolivia no está en la lista, se omite.
⏭️ País Bosnia-and-Herzegovina no está en la lista, se omite.
⏭️ País Botswana no está en la lista, se omite.
✅ Datos extraídos correctamente para Brazil
⏭️ País Brunei no está en la lista, se omite.
✅ Datos extraídos correctamente para Bulgaria
⏭️ País Cameroon no está en la lista, se omite.
✅ Datos extraídos correctamente para Canada
⏭️ País Cape-Verde no está en la lista, se omite.
✅ Datos extraídos correctamente para Chile
✅ Da

In [28]:
# obtain paraguay url
template_url = country_links[0]
country = "Paraguay"

# Separar por "/"
parts = template_url.strip("/").split("/")

# Reemplazar el país (posición 3: después del dominio)

parts[3] = country

# Reconstruir la URL
custom_url = "/".join(parts) + "/"

In [29]:
# extract and include paraguay

response = requests.get(custom_url, headers=headers)
html = response.text

match = re.search(pattern, html, re.DOTALL)
if match:
    data_str = match.group(1)
    data = json.loads(data_str)

    df = pd.DataFrame(data[1:], columns=data[0])
    df.columns = ["Period", "Value"]
    df["Country"] = country

    df_pivot = df.pivot(index="Period", columns="Country", values="Value")
    master_df = pd.merge(master_df, df_pivot, how="outer", left_index=True, right_index=True)

    print(f"✅ Datos extraídos correctamente para {country}")

✅ Datos extraídos correctamente para Paraguay


In [31]:
# Resetear el índice para separar "Period"
master_df = master_df.reset_index()

In [32]:
# Separar "Q1 1990" en "Quarter" y "Year"
master_df[["Quarter", "Year"]] = master_df["Period"].str.extract(r"Q(\d)\s+(\d{4})").astype(int)

In [33]:
# Reordenar las columnas: primero Quarter y Year
cols = ["Quarter", "Year"] + [col for col in master_df.columns if col not in ["Period", "Quarter", "Year"]]
master_df = master_df[cols]

In [34]:
# Ordenar por Year y Quarter
master_df = master_df.sort_values(by=["Year", "Quarter"]).reset_index(drop=True)

In [35]:
master_df.head()

,Quarter,Year,Australia,Austria,Belgium,Brazil,Bulgaria,Canada,Chile,Colombia,...,South-Africa,South-Korea,Spain,Sweden,Switzerland,Thailand,Turkey,USA,United-Kingdom,Paraguay
0,1,1961,1.98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,23.29,NaN,NaN,NaN,NaN,NaN,-5.16,15.46,NaN
1,2,1961,-1.67,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,12.22,NaN,NaN,NaN,NaN,NaN,-1.91,15.49,NaN
2,3,1961,1.22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,26.66,NaN,NaN,NaN,NaN,NaN,2.68,13.40,NaN
3,4,1961,-3.37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,50.41,NaN,NaN,NaN,NaN,NaN,6.23,6.95,NaN
4,1,1962,0.41,NaN,NaN,NaN,NaN,6.56,NaN,NaN,...,NaN,25.56,NaN,NaN,NaN,NaN,NaN,9.64,3.55,NaN


In [36]:
master_df.tail()

,Quarter,Year,Australia,Austria,Belgium,Brazil,Bulgaria,Canada,Chile,Colombia,...,South-Africa,South-Korea,Spain,Sweden,Switzerland,Thailand,Turkey,USA,United-Kingdom,Paraguay
253,2,2024,5.04,0.54,2.71,9.21,0.76,2.08,5.66,3.23,...,0.55,1.06,3.59,2.29,0.06,-5.44,73.99,5.59,3.19,9.12
254,3,2024,5.63,2.25,-0.95,15.82,5.01,2.56,7.34,23.91,...,1.19,0.61,3.74,2.73,-0.19,6.34,44.34,5.93,6.20,9.06
255,4,2024,5.54,1.74,7.57,15.61,9.27,6.53,11.37,45.43,...,0.12,0.74,8.72,3.82,1.07,4.92,45.85,4.54,4.70,12.06
256,1,2025,5.85,1.38,4.23,16.50,11.23,5.13,7.81,18.72,...,-1.34,-1.58,7.33,-1.19,1.66,5.13,30.21,4.87,7.14,17.00
257,2,2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,6.92,NaN,NaN,NaN,NaN,3.88,NaN,NaN


In [42]:
# Guardar como archivo Parquet
master_df.to_parquet("data/investment_growth_values.parquet", index=False)